In [7]:
#pip install python-binance

In [1]:
import pandas as pd
import math
import os.path
import time
from binance.client import Client
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)

### API
binance_api_key = 'pAtN761JV67xkRYnF4hI8RHL0DmInhHRoV50yPpz6VijEilHQdiHL7WIa4Ho60Om'    #Enter your own API-key here
binance_api_secret = 'o3juMWdjvpOhI1jfhHKwN6uereY31BSvwsDHaWWF6ytO150Y4xV1Ig3yWCtdaWwW' #Enter your own API-secret here

### CONSTANTS
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)


### FUNCTIONS
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    return old, new

def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'): print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: data_df = data
    data_df.set_index('timestamp', inplace=True)
    if save: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

In [2]:
data = get_all_binance('BTCUSDT','1d', save=False)
data.reset_index(level=0, inplace=True)

All caught up..!


In [3]:
data.head()

,timestamp,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
0,2017-08-17,4261.48000000,4485.39000000,4200.74000000,4285.08000000,795.15037700,1503014399999,3454770.05073206,3427,616.24854100,2678216.40060401,8733.91139481
1,2017-08-18,4285.08000000,4371.52000000,3938.77000000,4108.37000000,1199.88826400,1503100799999,5086958.30617151,5233,972.86871000,4129123.31651808,9384.14140858
2,2017-08-19,4108.37000000,4184.69000000,3850.00000000,4139.98000000,381.30976300,1503187199999,1549483.73542151,2153,274.33604200,1118001.87008735,9184.08552906
3,2017-08-20,4120.98000000,4211.08000000,4032.62000000,4086.29000000,467.08302200,1503273599999,1930364.39032646,2321,376.79594700,1557401.33373730,10125.41408414
4,2017-08-21,4069.13000000,4119.62000000,3911.79000000,4016.00000000,691.74306000,1503359999999,2797231.71402728,3972,557.35610700,2255662.55315837,11706.76997007


In [16]:
import matplotlib.pyplot as plt
import numpy as np

In [4]:
data[-2:-1]

,timestamp,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
809,2019-11-04,9196.46000000,9513.68000000,9115.84000000,9393.35000000,45894.45627700,1572911999999,426792003.17490647,453749,24741.81592500,230142613.20752436,0


In [53]:
np.array(data[-2:-1])[0][0]

Timestamp('2019-10-31 00:00:00')

In [55]:
resultjson = {
    'DATE_INFO': np.array(data[-2:-1])[0][0],
    'CLOSE': np.array(data[-2:-1])[0][4],
    'OPENING': np.array(data[-2:-1])[0][1],
    'MAX_VALUE': np.array(data[-2:-1])[0][2],
    'MIN_VALUE': np.array(data[-2:-1])[0][3],
    'VOL': np.array(data[-2:-1])[0][5],
    'VAR': np.nan
}

In [56]:
resultjson

{'DATE_INFO': Timestamp('2019-10-31 00:00:00'),
 'CLOSE': '9140.85000000',
 'OPENING': '9154.02000000',
 'MAX_VALUE': '9405.00000000',
 'MIN_VALUE': '8913.00000000',
 'VOL': '54376.02490200',
 'VAR': nan}

In [29]:
trades = binance_client.get_recent_trades(symbol='BCHABCBTC')

In [32]:
np.float(pd.DataFrame(trades)['price'][0])

NameError: name 'np' is not defined

In [ ]:
{'bitcoin': 'BTCUSDT', 'ethereum': 'ETHUSDT', 'ripple': 'XRPUSDT', 'litecoin': 'LTCUSDT',
'iota': 'IOTABTC', 'bitcoincash': 'BCHABCBTC'}

In [14]:
data[-2:-1]["timestamp"]

805   2019-10-31
Name: timestamp, dtype: datetime64[ns]

In [42]:
resultjson= {}

In [43]:
for i in range(3):
    resultjson[i]={'a':1,'b':2}

In [44]:
resultjson

{0: {'a': 1, 'b': 2}, 1: {'a': 1, 'b': 2}, 2: {'a': 1, 'b': 2}}

In [45]:
resultjson[0]['a']

1